TO DO:
- Implement differences between efficiency/power curve PSDs
- Update populated curve PSD to reflect Diameter units (Cell B3), Power (B4), Pwr/Eta cell (B5) 

##### Imports, File Setup

In [27]:
#Imports
import os
import pandas as pd
import numpy as np
from openpyxl import load_workbook

In [28]:
# Dictionaries
curve_dict = {
    'Flow':'Q',
    'Head':'H',
    'Power':'P2',
    'Efficiency':'Eta2',
    'NPSH':'NPSH'
}

In [31]:
# This is the folder/file with the curve export csv
myDir = r"C:\Users\104092\OneDrive - Grundfos\Documents\1 - PROJECTS\SPE Integration\SPE_PumpCurve"
myFile = "PumpCurves.csv"
filePath = os.path.join(myDir, myFile)

##### Function Definitions and Class Definitions

In [29]:
# Affinity Laws function
def apply_affinity_laws(row, n2, n1):
    N1, N2 = n1, n2
    Q2 = row['Q'] * (N2 / N1)
    H2 = row['H'] * (N2 / N1)**2
    NPSH2 = row['NPSH'] * (N2 / N1)**2
    P22 = row['P2'] * (N2 / N1)**3
    # P12 = row['P1'] * (N2 / N1)**3
    return pd.Series([Q2, H2, P22, NPSH2], index=['Q','H','P2','NPSH'])

In [30]:
# Create class to hold each curve by product number/trim
class Curve:
    def __init__(self, pumpModel: str, dataframe):
        self.name = pumpModel
        self.df = dataframe
        self.pn = dataframe.iloc[0]['ProductNumber']
        self.frequency = dataframe.iloc[0]['Frequency']
        self.phase = dataframe.iloc[0]['Phase']
        self.trims = self.df['RPM(Curve nominal)'].unique().tolist()
        self.trim_curves = {} 


    def __repr__(self) -> str:
        return f"Pump Model(name={self.name}, dataframe=\n{self.df})"


    def return_xy_points(self, curveType:str):
        '''
        Returns 2 lists: 1 - flow values, 2 - Head or Power or NPSH
        
        Parameters:
        curveType (str): The list of y-values. Available options are:
            - 'Head'
            - 'Power'
            - 'NPSH'
        
        Raises:
        ValueError: If the action is not one of the available options.
        '''
        curve_types = ['Head','Power','NPSH']

        if curveType not in curve_types:
            raise ValueError(f"Invalid Curve Type. Available options are: {', '.join(curve_types)}")
        else:
            x_values = self.df['Q'].tolist()
            y_values = self.df[curve_dict[curveType]].tolist()
            return x_values, y_values
    

    def create_grouped_trim_curves(self):
        '''Group entire curve df by the trim/speed column'''
        grouped = self.df.groupby('RPM(Curve nominal)')
        for group_trim, trim_df in grouped:
            self.trim_curves[group_trim] = trim_df[['Q','H','P2','NPSH']]


    def create_new_trim_df(self, n2):
        """
        Takes in speed n2 and applies affinity laws to max available existing trim to calculate new curve data

        Output: Adds a dataframe to self.trim_curves dictionary

        """
        # Finds max existing trim and uses that as n1
        n1 = self.max_trim
        df1 = self.trim_curves[n1]

        # Apply the function to each row of df1 to create df2
        df2 = df1.apply(apply_affinity_laws, axis=1, args=(n2,n1))

        # Add new dataframe to dictionary trim_curves
        self.trim_curves.update({n2:df2}) 


    @property
    def max_trim(self):
        return max(self.trims)

##### Create objects from GPI Curve Export

In [32]:
# This separates the GPI curve export by groups according to ProductNumber
df= pd.read_csv(filePath, sep=";", index_col=False, skip_blank_lines=False) # FOR SPE
df = df.replace(',','.', regex=True)

# Drop NaN rows before grouping
df_non_nan = df.dropna(how='all')

# Forward fill productname and curve nominal columns for grouping 
df_non_nan['Product name'] = df_non_nan['Product name'].ffill()
df_non_nan['RPM(Curve nominal)'] = df_non_nan['RPM(Curve nominal)'].ffill()

# Group by the pump model column
grouped = df_non_nan.groupby('Product name')

# Make a model from each group 
group_objects = {}
for pumpModel, group_df in grouped:
    group_object = Curve(pumpModel=pumpModel, dataframe=group_df)
    group_object.create_grouped_trim_curves()
    group_objects[(f'{pumpModel}')] = group_object
    

C:\Users\104092\AppData\Local\Temp\ipykernel_89188\2750130057.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_nan['Product name'] = df_non_nan['Product name'].ffill()
C:\Users\104092\AppData\Local\Temp\ipykernel_89188\2750130057.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_non_nan['RPM(Curve nominal)'] = df_non_nan['RPM(Curve nominal)'].ffill()


##### Creates new speed curve data using affinity laws

In [33]:
# Iterate through list of models and and create new speed dataframes at n2
n2 = 1800

for object_name in group_objects:
    model_object = group_objects[object_name]
    model_object.create_new_trim_df(n2)

##### Writes curve data to excel file

In [34]:
# Combine DataFrames from the dictionary into a single DataFrame
def combine_dataframes(dict_of_dfs):
    combined_df = pd.DataFrame()
    for speed, df in dict_of_dfs.items():
        # Add a column for the speed
        df['Speed'] = speed
        # Append to the combined DataFrame
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    return combined_df

# Write to excel for data validation, or plot with matplotlib
# Create Excel Sheet Tabs, 1 for each curve
output_file = 'SPE Added Minimum Speed Curves.xlsx'

with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    # Loop through each curve object
    for object_name in group_objects:
        curve_obj = group_objects[object_name]
        combined_df = combine_dataframes(curve_obj.trim_curves)
            
        # Write the combined DataFrame to an Excel sheet
        combined_df.to_excel(writer, sheet_name=object_name, index=False)   

In [ ]:
# Leverage Luke's code to write these to xml for deployment

